# 02 Handle different source data on the example of weather data

I like to practise analysing different source data in different file formats, determine differences and join them together to a single output. Weather data is free and available in many formats (json, csv, xml etc.) so that I consider this being a nice example.

**Learnings:**
* requesting APIs (JSON response); if neccessary, use `stream=True`and `iter_content()`
* pretty print json with `json.dumps()`
* `pd.json_normalize()` to create DataFrame from json
* DataFrame processing with `df[ df[condition_col] == condition_value, change_col] = value`
* `merge()` `DataFrameGoupBy` back into original DataFrame
* Simulate SUM(col) OVER(ORDER BY group_col1 PARTITION BY group_col2) with pandas 

## 1 Load source data

When the data in `requests.request("GET", url, params=querystring)` is too large for memory, it should be done with an additional `stream` argument within in the function call:
```py
with requests.get(url, stream=True) as r:
    r.raise_for_status() #shows html error if arises
    for chunk in r.iter_content(chunk_size=8192): #in bytes (1MB = million iters)
        #or r.iter_lines() #parses line by line
        tmp = pd.json_normalize(chunk, *kwargs) #process that json chunk
        df.append(tmp)
```
   
## 1.1 Open Weather Map

In [279]:
import requests
import os

url = "https://api.openweathermap.org/data/2.5/onecall"
#Leipzig in [51.3396, 12.3713]
querystring = {"lat":"51.3396", "lon": "12.3713", "exclude": "current,minutely", "units":"metric", "appid":OWM_KEY} #parameters

headers = {} #TODO what are headers for? 

In [172]:
response_owm = requests.request("GET", url, params=querystring)
print(response_owm.text)

{"lat":51.3396,"lon":12.3713,"timezone":"Europe/Berlin","timezone_offset":7200,"hourly":[{"dt":1626184800,"temp":29.47,"feels_like":29.42,"pressure":1006,"humidity":43,"dew_point":15.59,"uvi":4.16,"clouds":20,"visibility":10000,"wind_speed":4.18,"wind_deg":77,"wind_gust":5.65,"weather":[{"id":801,"main":"Clouds","description":"few clouds","icon":"02d"}],"pop":0},{"dt":1626188400,"temp":29.19,"feels_like":29.43,"pressure":1006,"humidity":46,"dew_point":16.39,"uvi":2.71,"clouds":28,"visibility":10000,"wind_speed":4.42,"wind_deg":63,"wind_gust":5.61,"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"pop":0},{"dt":1626192000,"temp":28.53,"feels_like":29.07,"pressure":1006,"humidity":50,"dew_point":17.1,"uvi":1.25,"clouds":40,"visibility":10000,"wind_speed":4.31,"wind_deg":57,"wind_gust":7.27,"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03d"}],"pop":0.17},{"dt":1626195600,"temp":27.43,"feels_like":28.39,"pressure":1005,"hum

In [131]:
'''
url = "https://api.openweathermap.org/data/2.5/onecall?lat=33.44&lon=-94.04&exclude=hourly,daily&appid=OMW_KEY"
with requests.get(url, stream=True) as r:
    r.raise_for_status() #shows html error if arises
    for chunk in r.iter_content(chunk_size=100): #in bytes (1MB = million iters)
        #or r.iter_lines() #parses line by line
        print(chunk)
'''

'\nurl = "https://api.openweathermap.org/data/2.5/onecall?lat=33.44&lon=-94.04&exclude=hourly,daily&appid=8c159bf63b964af1e78ca3be1ff3e86d"\nwith requests.get(url, stream=True) as r:\n    r.raise_for_status() #shows html error if arises\n    for chunk in r.iter_content(chunk_size=100): #in bytes (1MB = million iters)\n        #or r.iter_lines() #parses line by line\n        print(chunk)\n'

## 1.2 Weather API

In [293]:
url = f"http://api.weatherapi.com/v1/forecast.json?key={os.environ['WAPI_KEY']}&q=Leipzig&days=7&aqi=no&alerts=no"
print(url)
response_wapi = requests.request("GET", url)
print(response_wapi.text)

http://api.weatherapi.com/v1/forecast.json?key=1ce05a0c63b54afc88495153211307&q=Leipzig&days=7&aqi=no&alerts=no
{"location":{"name":"Leipzig","region":"Sachsen","country":"Germany","lat":51.3,"lon":12.33,"tz_id":"Europe/Berlin","localtime_epoch":1626193910,"localtime":"2021-07-13 18:31"},"current":{"last_updated_epoch":1626192900,"last_updated":"2021-07-13 18:15","temp_c":27.0,"temp_f":80.6,"is_day":1,"condition":{"text":"Sunny","icon":"//cdn.weatherapi.com/weather/64x64/day/113.png","code":1000},"wind_mph":6.9,"wind_kph":11.2,"wind_degree":60,"wind_dir":"ENE","pressure_mb":1007.0,"pressure_in":30.2,"precip_mm":0.0,"precip_in":0.0,"humidity":51,"cloud":0,"feelslike_c":27.9,"feelslike_f":82.1,"vis_km":10.0,"vis_miles":6.0,"uv":7.0,"gust_mph":11.4,"gust_kph":18.4},"forecast":{"forecastday":[{"date":"2021-07-13","date_epoch":1626134400,"day":{"maxtemp_c":27.9,"maxtemp_f":82.2,"mintemp_c":16.5,"mintemp_f":61.7,"avgtemp_c":22.2,"avgtemp_f":72.0,"maxwind_mph":9.8,"maxwind_kph":15.8,"totalpre

## 1.3 Load in pandas

In [26]:
import pandas as pd
import json

### 1.3.1 Weather API

In [105]:
wapi_json = json.loads(response_wapi.text) #returns dict
print(json.dumps(wapi_json, indent=2)) #returns str (pretty print)
wapi_json.keys() #dict_keys(['location', 'current', 'forecast'])

dict_keys(['location', 'current', 'forecast'])

In [97]:
wapi_fc_day_df = pd.json_normalize(wapi_json["forecast"]["forecastday"])
print(wapi_fc_day_df.columns)
wapi_fc_day_df

Index(['date', 'date_epoch', 'hour', 'day.maxtemp_c', 'day.maxtemp_f',
       'day.mintemp_c', 'day.mintemp_f', 'day.avgtemp_c', 'day.avgtemp_f',
       'day.maxwind_mph', 'day.maxwind_kph', 'day.totalprecip_mm',
       'day.totalprecip_in', 'day.avgvis_km', 'day.avgvis_miles',
       'day.avghumidity', 'day.daily_will_it_rain', 'day.daily_chance_of_rain',
       'day.daily_will_it_snow', 'day.daily_chance_of_snow',
       'day.condition.text', 'day.condition.icon', 'day.condition.code',
       'day.uv', 'astro.sunrise', 'astro.sunset', 'astro.moonrise',
       'astro.moonset', 'astro.moon_phase', 'astro.moon_illumination'],
      dtype='object')


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,...,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination
0,2021-07-13,1626134400,"[{'time_epoch': 1626127200, 'time': '2021-07-1...",27.9,82.2,16.5,61.7,22.2,72.0,9.8,...,Partly cloudy,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,7.0,05:10 AM,09:22 PM,08:30 AM,11:36 PM,First Quarter,34
1,2021-07-14,1626220800,"[{'time_epoch': 1626213600, 'time': '2021-07-1...",21.8,71.2,15.6,60.1,18.9,66.0,13.9,...,Moderate rain,//cdn.weatherapi.com/weather/64x64/day/302.png,1189,4.0,05:11 AM,09:21 PM,09:48 AM,11:54 PM,First Quarter,41
2,2021-07-15,1626307200,"[{'time_epoch': 1626300000, 'time': '2021-07-1...",23.8,74.8,13.5,56.3,18.9,66.0,11.0,...,Partly cloudy,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,4.0,05:12 AM,09:20 PM,11:05 AM,No moonset,First Quarter,48


In [103]:
#wapi_fc_hr_df = pd.json_normalize(list(wapi_day_df['hour'][0]))
wapi_fc_hr_df = pd.json_normalize(wapi_json["forecast"]["forecastday"],
                                  record_path=['hour'],
                                  meta=['date'],)
print(wapi_fc_hr_df.columns)
wapi_fc_hr_df.sample(5)

Index(['time_epoch', 'time', 'temp_c', 'temp_f', 'is_day', 'wind_mph',
       'wind_kph', 'wind_degree', 'wind_dir', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feelslike_c',
       'feelslike_f', 'windchill_c', 'windchill_f', 'heatindex_c',
       'heatindex_f', 'dewpoint_c', 'dewpoint_f', 'will_it_rain',
       'chance_of_rain', 'will_it_snow', 'chance_of_snow', 'vis_km',
       'vis_miles', 'gust_mph', 'gust_kph', 'uv', 'condition.text',
       'condition.icon', 'condition.code', 'date'],
      dtype='object')


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,...,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.text,condition.icon,condition.code,date
28,1626228000,2021-07-14 04:00,17.4,63.3,0,4.5,7.2,214,SW,1008.0,...,0,10.0,6.0,7.4,11.9,1.0,Light rain shower,//cdn.weatherapi.com/weather/64x64/night/353.png,1240,2021-07-14
20,1626199200,2021-07-13 20:00,23.4,74.1,1,7.4,11.9,57,ENE,1007.0,...,0,10.0,6.0,15.4,24.8,5.0,Patchy rain possible,//cdn.weatherapi.com/weather/64x64/day/176.png,1063,2021-07-13
36,1626256800,2021-07-14 12:00,20.5,68.9,1,12.5,20.2,193,SSW,1010.0,...,0,10.0,6.0,15.7,25.2,5.0,Light rain shower,//cdn.weatherapi.com/weather/64x64/day/353.png,1240,2021-07-14
45,1626289200,2021-07-14 21:00,16.8,62.2,1,6.9,11.2,232,SW,1012.0,...,0,10.0,6.0,13.2,21.2,5.0,Partly cloudy,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,2021-07-14
15,1626181200,2021-07-13 15:00,27.5,81.5,1,9.2,14.8,71,ENE,1007.0,...,0,10.0,6.0,11.4,18.4,7.0,Partly cloudy,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,2021-07-13


### 1.3.2 Open Weather Map

In [173]:
owm_json = json.loads(response_owm.text)
print(json.dumps(owm_json, indent=2))

{
  "lat": 51.3396,
  "lon": 12.3713,
  "timezone": "Europe/Berlin",
  "timezone_offset": 7200,
  "hourly": [
    {
      "dt": 1626184800,
      "temp": 29.47,
      "feels_like": 29.42,
      "pressure": 1006,
      "humidity": 43,
      "dew_point": 15.59,
      "uvi": 4.16,
      "clouds": 20,
      "visibility": 10000,
      "wind_speed": 4.18,
      "wind_deg": 77,
      "wind_gust": 5.65,
      "weather": [
        {
          "id": 801,
          "main": "Clouds",
          "description": "few clouds",
          "icon": "02d"
        }
      ],
      "pop": 0
    },
    {
      "dt": 1626188400,
      "temp": 29.19,
      "feels_like": 29.43,
      "pressure": 1006,
      "humidity": 46,
      "dew_point": 16.39,
      "uvi": 2.71,
      "clouds": 28,
      "visibility": 10000,
      "wind_speed": 4.42,
      "wind_deg": 63,
      "wind_gust": 5.61,
      "weather": [
        {
          "id": 802,
          "main": "Clouds",
          "description": "scattered clouds",
       

In [159]:
owm_json.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'hourly', 'daily'])

In [174]:
owm_fc_day_df = pd.json_normalize(owm_json,
                                 record_path=['daily'],
                                 meta=['lat', 'lon'])
print(owm_fc_day_df.columns)
owm_fc_day_df.sample(5)

Index(['dt', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase',
       'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg',
       'wind_gust', 'weather', 'clouds', 'pop', 'rain', 'uvi', 'temp.day',
       'temp.min', 'temp.max', 'temp.night', 'temp.eve', 'temp.morn',
       'feels_like.day', 'feels_like.night', 'feels_like.eve',
       'feels_like.morn', 'lat', 'lon'],
      dtype='object')


,dt,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,...,temp.max,temp.night,temp.eve,temp.morn,feels_like.day,feels_like.night,feels_like.eve,feels_like.morn,lat,lon
2,1626346800,1626318701,1626376852,1626339900,0,0.18,1012,62,13.83,4.32,...,21.87,15.54,19.95,15.57,21.41,15.45,19.91,15.35,51.3396,12.3713
3,1626433200,1626405173,1626463192,1626431040,1626387000,0.21,1018,59,14.60,2.86,...,24.10,18.69,22.84,16.01,22.91,18.63,22.96,15.94,51.3396,12.3713
4,1626519600,1626491646,1626549529,1626522300,1626474360,0.25,1018,54,16.89,4.28,...,27.05,20.07,23.24,19.53,27.72,20.53,23.63,19.65,51.3396,12.3713
1,1626260400,1626232231,1626290509,1626248880,1626299640,0.14,1009,67,14.54,5.70,...,22.35,15.08,20.65,16.42,20.87,14.97,20.65,16.47,51.3396,12.3713
5,1626606000,1626578121,1626635864,1626613680,1626561780,0.29,1018,58,18.15,6.42,...,27.28,19.14,23.68,20.44,28.28,18.99,23.77,20.86,51.3396,12.3713


In [161]:
pd.json_normalize(owm_fc_day_df.weather[0])

,id,main,description,icon
0,502,Rain,heavy intensity rain,10d


In [176]:
owm_fc_hr_df = pd.json_normalize(owm_json,
                                 record_path=['hourly'],
                                 meta=['lat', 'lon'])
owm_fc_hr_df

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop,rain.1h,lat,lon
0,1626184800,29.47,29.42,1006,43,15.59,4.16,20,10000,4.18,77,5.65,"[{'id': 801, 'main': 'Clouds', 'description': ...",0.00,NaN,51.3396,12.3713
1,1626188400,29.19,29.43,1006,46,16.39,2.71,28,10000,4.42,63,5.61,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.00,NaN,51.3396,12.3713
2,1626192000,28.53,29.07,1006,50,17.10,1.25,40,10000,4.31,57,7.27,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.17,NaN,51.3396,12.3713
3,1626195600,27.43,28.39,1005,57,18.16,0.55,54,10000,3.91,53,8.48,"[{'id': 803, 'main': 'Clouds', 'description': ...",0.30,NaN,51.3396,12.3713
4,1626199200,25.08,25.39,1005,67,18.53,0.17,69,10000,3.87,45,9.13,"[{'id': 500, 'main': 'Rain', 'description': 'l...",0.39,0.17,51.3396,12.3713
5,1626202800,22.17,22.56,1005,81,18.77,0.00,100,10000,2.61,65,5.93,"[{'id': 501, 'main': 'Rain', 'description': 'm...",0.94,1.49,51.3396,12.3713
6,1626206400,20.83,21.37,1006,92,19.47,0.00,100,3579,3.23,185,7.26,"[{'id': 502, 'main': 'Rain', 'description': 'h...",1.00,5.03,51.3396,12.3713
7,1626210000,19.84,20.41,1006,97,19.27,0.00,100,10000,0.25,240,4.25,"[{'id': 502, 'main': 'Rain', 'description': 'h...",1.00,5.71,51.3396,12.3713
8,1626213600,18.53,18.97,1007,97,18.03,0.00,100,9194,3.00,94,5.25,"[{'id': 502, 'main': 'Rain', 'description': 'h...",1.00,4.44,51.3396,12.3713
9,1626217200,17.95,18.28,1007,95,17.13,0.00,100,10000,3.34,174,6.32,"[{'id': 500, 'main': 'Rain', 'description': 'l...",1.00,0.98,51.3396,12.3713


In [163]:
from time import ctime
as_time=pd.DataFrame(map(ctime, owm_fc_hr_df["dt"]))
pd.to_datetime(as_time[0])

0    2021-07-13 16:00:00
1    2021-07-13 17:00:00
2    2021-07-13 18:00:00
3    2021-07-13 19:00:00
4    2021-07-13 20:00:00
5    2021-07-13 21:00:00
6    2021-07-13 22:00:00
7    2021-07-13 23:00:00
8    2021-07-14 00:00:00
9    2021-07-14 01:00:00
10   2021-07-14 02:00:00
11   2021-07-14 03:00:00
12   2021-07-14 04:00:00
13   2021-07-14 05:00:00
14   2021-07-14 06:00:00
15   2021-07-14 07:00:00
16   2021-07-14 08:00:00
17   2021-07-14 09:00:00
18   2021-07-14 10:00:00
19   2021-07-14 11:00:00
20   2021-07-14 12:00:00
21   2021-07-14 13:00:00
22   2021-07-14 14:00:00
23   2021-07-14 15:00:00
24   2021-07-14 16:00:00
25   2021-07-14 17:00:00
26   2021-07-14 18:00:00
27   2021-07-14 19:00:00
28   2021-07-14 20:00:00
29   2021-07-14 21:00:00
30   2021-07-14 22:00:00
31   2021-07-14 23:00:00
32   2021-07-15 00:00:00
33   2021-07-15 01:00:00
34   2021-07-15 02:00:00
35   2021-07-15 03:00:00
36   2021-07-15 04:00:00
37   2021-07-15 05:00:00
38   2021-07-15 06:00:00
39   2021-07-15 07:00:00


## 1.4 Process DataFrames

In [177]:
print(owm_fc_hr_df.info())
print(owm_fc_hr_df.describe().T)
owm_fc_hr_df = owm_fc_hr_df[["dt", "temp", "feels_like", "clouds", "wind_speed", "rain.1h"]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dt          48 non-null     int64  
 1   temp        48 non-null     float64
 2   feels_like  48 non-null     float64
 3   pressure    48 non-null     int64  
 4   humidity    48 non-null     int64  
 5   dew_point   48 non-null     float64
 6   uvi         48 non-null     float64
 7   clouds      48 non-null     int64  
 8   visibility  48 non-null     int64  
 9   wind_speed  48 non-null     float64
 10  wind_deg    48 non-null     int64  
 11  wind_gust   48 non-null     float64
 12  weather     48 non-null     object 
 13  pop         48 non-null     float64
 14  rain.1h     6 non-null      float64
 15  lat         48 non-null     object 
 16  lon         48 non-null     object 
dtypes: float64(8), int64(6), object(3)
memory usage: 6.5+ KB
None
            count          mean           std      

In [168]:
print(wapi_fc_hr_df.info())
print(wapi_fc_hr_df.describe().T)
wapi_fc_hr_df = wapi_fc_hr_df[["time_epoch", "time", "date", "temp_c", "wind_kph", 
                               "precip_mm", "feelslike_c", "will_it_rain", "chance_of_rain"]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   time_epoch      72 non-null     int64  
 1   time            72 non-null     object 
 2   temp_c          72 non-null     float64
 3   temp_f          72 non-null     float64
 4   is_day          72 non-null     int64  
 5   wind_mph        72 non-null     float64
 6   wind_kph        72 non-null     float64
 7   wind_degree     72 non-null     int64  
 8   wind_dir        72 non-null     object 
 9   pressure_mb     72 non-null     float64
 10  pressure_in     72 non-null     float64
 11  precip_mm       72 non-null     float64
 12  precip_in       72 non-null     float64
 13  humidity        72 non-null     int64  
 14  cloud           72 non-null     int64  
 15  feelslike_c     72 non-null     float64
 16  feelslike_f     72 non-null     float64
 17  windchill_c     72 non-null     float

In [169]:
wapi_fc_hr_df.head(5)

,time_epoch,time,date,temp_c,wind_kph,precip_mm,feelslike_c,will_it_rain,chance_of_rain
0,1626127200,2021-07-13 00:00,2021-07-13,17.7,7.9,0.0,17.7,0,0
1,1626130800,2021-07-13 01:00,2021-07-13,17.1,6.5,0.0,17.1,0,0
2,1626134400,2021-07-13 02:00,2021-07-13,16.9,6.5,0.0,16.9,0,0
3,1626138000,2021-07-13 03:00,2021-07-13,16.7,6.1,0.0,16.7,0,0
4,1626141600,2021-07-13 04:00,2021-07-13,16.5,6.1,0.0,16.5,0,0


In [178]:
owm_fc_hr_df.head(5)

,dt,temp,feels_like,clouds,wind_speed,rain.1h
0,1626184800,29.47,29.42,20,4.18,NaN
1,1626188400,29.19,29.43,28,4.42,NaN
2,1626192000,28.53,29.07,40,4.31,NaN
3,1626195600,27.43,28.39,54,3.91,NaN
4,1626199200,25.08,25.39,69,3.87,0.17


### 1.4 Join Data

In [207]:
weather_df= wapi_fc_hr_df.set_index("time_epoch").join(owm_fc_hr_df.set_index("dt"), how="outer", sort=True)

In [208]:
#weather_df.loc[:, weather_df["temp"].notna(), ["rain.1h"]]=weather_df[ weather_df["temp"].notna() ]["rain.1h"].fillna(0)

weather_df.loc[weather_df["temp"].notna() & weather_df["rain.1h"].isna(), "rain.1h"]=0

In [209]:
weather_df[ weather_df["temp"].notna() ]

,time,date,temp_c,wind_kph,precip_mm,feelslike_c,will_it_rain,chance_of_rain,temp,feels_like,clouds,wind_speed,rain.1h
1626184800,2021-07-13 16:00,2021-07-13,27.9,14.4,0.0,28.9,0,0,29.47,29.42,20.0,4.18,0.00
1626188400,2021-07-13 17:00,2021-07-13,26.6,13.7,0.0,27.8,0,0,29.19,29.43,28.0,4.42,0.00
1626192000,2021-07-13 18:00,2021-07-13,25.3,13.0,0.0,26.8,0,0,28.53,29.07,40.0,4.31,0.00
1626195600,2021-07-13 19:00,2021-07-13,24.0,12.2,0.0,25.7,0,0,27.43,28.39,54.0,3.91,0.00
1626199200,2021-07-13 20:00,2021-07-13,23.4,11.9,0.0,25.3,0,21,25.08,25.39,69.0,3.87,0.17
1626202800,2021-07-13 21:00,2021-07-13,22.7,11.5,0.1,25.0,0,42,22.17,22.56,100.0,2.61,1.49
1626206400,2021-07-13 22:00,2021-07-13,22.1,11.2,0.1,24.6,0,62,20.83,21.37,100.0,3.23,5.03
1626210000,2021-07-13 23:00,2021-07-13,20.8,10.8,2.0,20.8,0,66,19.84,20.41,100.0,0.25,5.71
1626213600,2021-07-14 00:00,2021-07-14,19.4,10.4,3.9,19.4,1,70,18.53,18.97,100.0,3.00,4.44
1626217200,2021-07-14 01:00,2021-07-14,18.1,10.1,5.8,18.1,1,74,17.95,18.28,100.0,3.34,0.98


### 1.5 Calculations (groupby, merge)

Here I want to simulate SQL window functions. It can be done by 
1. `DataFrame.groupby()` (Order by in SQL)
2. `DataFrameGroupBy.aggregation()` (Apply any aggregation function (sum, mean etc.))
3. For cumulation: Return to 1. and apply `groupby(level=0).cumsum()`
2. `DataFrame.merge()` (Merge the aggregated values back)

In [253]:
day_group = weather_df.groupby("date").mean()["temp_c"]

Index(['2021-07-13', '2021-07-14', '2021-07-15'], dtype='object', name='date')

In [257]:
weather_df = weather_df.merge(day_group, 
                 left_on="date", 
                 right_index=True).rename(columns={"temp_c_y":"temp_avg_day"})

In [259]:
weather_df

,time,date,temp_c_x,wind_kph,precip_mm,feelslike_c,will_it_rain,chance_of_rain,temp,feels_like,clouds,wind_speed,rain.1h,temp_avg_day
1626127200,2021-07-13 00:00,2021-07-13,17.7,7.9,0.0,17.7,0,0,NaN,NaN,NaN,NaN,NaN,22.183333
1626130800,2021-07-13 01:00,2021-07-13,17.1,6.5,0.0,17.1,0,0,NaN,NaN,NaN,NaN,NaN,22.183333
1626134400,2021-07-13 02:00,2021-07-13,16.9,6.5,0.0,16.9,0,0,NaN,NaN,NaN,NaN,NaN,22.183333
1626138000,2021-07-13 03:00,2021-07-13,16.7,6.1,0.0,16.7,0,0,NaN,NaN,NaN,NaN,NaN,22.183333
1626141600,2021-07-13 04:00,2021-07-13,16.5,6.1,0.0,16.5,0,0,NaN,NaN,NaN,NaN,NaN,22.183333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626368400,2021-07-15 19:00,2021-07-15,21.4,7.6,0.0,21.4,0,0,NaN,NaN,NaN,NaN,NaN,18.866667
1626372000,2021-07-15 20:00,2021-07-15,19.9,8.3,0.0,19.9,0,0,NaN,NaN,NaN,NaN,NaN,18.866667
1626375600,2021-07-15 21:00,2021-07-15,18.5,8.6,0.0,18.5,0,0,NaN,NaN,NaN,NaN,NaN,18.866667
1626379200,2021-07-15 22:00,2021-07-15,17.0,9.4,0.0,17.0,0,0,NaN,NaN,NaN,NaN,NaN,18.866667


In [272]:
total_rain_per_day = weather_df.groupby(["date", "time"]).sum().groupby(level=0).cumsum()["precip_mm"]
total_rain_per_day

date        time            
2021-07-13  2021-07-13 00:00    0.0
            2021-07-13 01:00    0.0
            2021-07-13 02:00    0.0
            2021-07-13 03:00    0.0
            2021-07-13 04:00    0.0
                               ... 
2021-07-15  2021-07-15 19:00    0.0
            2021-07-15 20:00    0.0
            2021-07-15 21:00    0.0
            2021-07-15 22:00    0.0
            2021-07-15 23:00    0.0
Name: precip_mm, Length: 72, dtype: float64

In [275]:
weather_df = weather_df.merge(total_rain_per_day, 
                 left_on="time", 
                 right_on="time").rename(columns={"precip_mm_y":"total_rain_per_day"})
weather_df

,time,date,temp_c_x,wind_kph,precip_mm_x,feelslike_c,will_it_rain,chance_of_rain,temp,feels_like,clouds,wind_speed,rain.1h,temp_avg_day,total_rain_per_day,precip_mm
0,2021-07-13 00:00,2021-07-13,17.7,7.9,0.0,17.7,0,0,NaN,NaN,NaN,NaN,NaN,22.183333,0.0,0.0
1,2021-07-13 01:00,2021-07-13,17.1,6.5,0.0,17.1,0,0,NaN,NaN,NaN,NaN,NaN,22.183333,0.0,0.0
2,2021-07-13 02:00,2021-07-13,16.9,6.5,0.0,16.9,0,0,NaN,NaN,NaN,NaN,NaN,22.183333,0.0,0.0
3,2021-07-13 03:00,2021-07-13,16.7,6.1,0.0,16.7,0,0,NaN,NaN,NaN,NaN,NaN,22.183333,0.0,0.0
4,2021-07-13 04:00,2021-07-13,16.5,6.1,0.0,16.5,0,0,NaN,NaN,NaN,NaN,NaN,22.183333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2021-07-15 19:00,2021-07-15,21.4,7.6,0.0,21.4,0,0,NaN,NaN,NaN,NaN,NaN,18.866667,0.0,0.0
68,2021-07-15 20:00,2021-07-15,19.9,8.3,0.0,19.9,0,0,NaN,NaN,NaN,NaN,NaN,18.866667,0.0,0.0
69,2021-07-15 21:00,2021-07-15,18.5,8.6,0.0,18.5,0,0,NaN,NaN,NaN,NaN,NaN,18.866667,0.0,0.0
70,2021-07-15 22:00,2021-07-15,17.0,9.4,0.0,17.0,0,0,NaN,NaN,NaN,NaN,NaN,18.866667,0.0,0.0
